<a href="https://colab.research.google.com/github/saraires/Proyecto-Modelos1/blob/main/99_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extrayendo Información desde Kaggle

# **99 - modelo solución**
### Santiago Palacio Cárdenas
### Sarai Restrepo Rodríguez
### Natalia Bernal Gutiérrez

En este notebook desarrollamos un modelo de clasificación supervisada para predecir el rendimiento global de los estudiantes en las pruebas Saber Pro, utilizando técnicas avanzadas de ingeniería de características y un modelo CatBoost optimizado para datos categóricos. A lo largo del flujo, realizamos el procesamiento del conjunto de entrenamiento, entrenamos el modelo en fase de validación para evaluar su desempeño y finalmente construimos un modelo definitivo con todos los datos disponibles para generar el archivo de envío requerido por Kaggle. Este notebook documenta de manera clara cada paso del proceso para garantizar reproducibilidad, trazabilidad y comprensión del pipeline completo.


# **1. Extrayendo Información desde Kaggle**

En esta sección descargamos los archivos oficiales de la competencia directamente desde Kaggle usando la interfaz de línea de comandos, lo cual nos permite obtener las versiones más recientes del conjunto de entrenamiento, prueba y ejemplos de envío. Esta descarga garantiza que trabajemos con los datos proporcionados por la competencia sin modificaciones locales y mantiene la coherencia entre el entorno de trabajo y la plataforma de evaluación. Además, dejar explícito el comando de extracción permite que cualquier persona que abra el notebook pueda replicar el proceso fácilmente.

In [ ]:
# Descargar datos desde Kaggle (opcional, requiere kaggle.json)
import os, shutil
from pathlib import Path

def setup_kaggle_config():
    # Busca kaggle.json en carpeta actual o Descargas y lo copia a ~/.kaggle
    candidates = [
        Path('kaggle.json'),
        Path.home() / 'Downloads' / 'kaggle.json',
    ]
    dest_dir = Path.home() / '.kaggle'
    dest_dir.mkdir(exist_ok=True)
    for c in candidates:
        if c.exists():
            shutil.copy2(str(c), str(dest_dir / 'kaggle.json'))
            print(f"kaggle.json configurado en {dest_dir}")
            return True
    print("No se encontró kaggle.json. Coloca tu token en la carpeta del notebook o en Descargas.")
    return False

def kaggle_download():
    try:
        from kaggle.api.kaggle_api_extended import KaggleApi
    except Exception as e:
        print("Kaggle API no disponible. Asegúrate de haber instalado 'kaggle'.")
        return False
    if not setup_kaggle_config():
        return False
    try:
        api = KaggleApi(); api.authenticate()
        target = Path.home() / 'Downloads'
        comp = 'udea-ai-4-eng-20252-pruebas-saber-pro-colombia'
        print(f"Descargando archivos de la competencia a: {target}")
        api.competition_download_files(comp, path=str(target), quiet=False)
        print("Descarga completada.")
        return True
    except Exception as e:
        print(f"Fallo en descarga Kaggle: {e}")
        return False

_ = kaggle_download()

kaggle.json configurado en C:\Users\sarai\.kaggle
Descargando archivos de la competencia a: C:\Users\sarai\Downloads


100%|██████████| 29.9M/29.9M [00:03<00:00, 9.19MB/s]


Descarga completada.


In [ ]:
# Descomprimir archivos descargados de Kaggle de forma portable
import os, glob, zipfile

def extract_kaggle_zip():
    candidates = []
    downloads = os.path.join(os.path.expanduser('~'), 'Downloads')
    candidates += glob.glob(os.path.join('.', 'udea*pruebas*colombia*.zip'))
    candidates += glob.glob(os.path.join(downloads, 'udea*pruebas*colombia*.zip'))
    for z in candidates:
        try:
            target_dir = os.path.dirname(z)
            with zipfile.ZipFile(z, 'r') as zf:
                zf.extractall(target_dir)
            print(f"Extraído: {z} -> {target_dir}")
        except Exception as e:
            print(f"No se pudo extraer {z}: {e}")

extract_kaggle_zip()

Extraído: .\udea-ai-4-eng-20252-pruebas-saber-pro-colombia-publicleaderboard-2025-11-27T16_03_07.zip -> .
Extraído: .\udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip -> .
Extraído: C:\Users\sarai\Downloads\udea-ai-4-eng-20252-pruebas-saber-pro-colombia-publicleaderboard-2025-11-27T16_03_07.zip -> C:\Users\sarai\Downloads
Extraído: .\udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip -> .
Extraído: C:\Users\sarai\Downloads\udea-ai-4-eng-20252-pruebas-saber-pro-colombia-publicleaderboard-2025-11-27T16_03_07.zip -> C:\Users\sarai\Downloads
Extraído: C:\Users\sarai\Downloads\udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip -> C:\Users\sarai\Downloads
Extraído: C:\Users\sarai\Downloads\udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip -> C:\Users\sarai\Downloads


# **2. Importaciones e instalaciones necesarias**

En esta sección instalamos y cargamos todas las librerías esenciales que utilizaremos a lo largo del desarrollo del modelo. Primero instalamos CatBoost, un algoritmo altamente eficiente para manejar grandes cantidades de variables categóricas y adecuado para problemas de clasificación multiclase como el rendimiento Saber Pro. Luego importamos herramientas de manejo de datos como pandas y numpy, junto con funciones de scikit-learn para realizar particiones, métricas de evaluación y manejo de pools de datos. Finalmente, cargamos librerías de visualización como matplotlib y seaborn, que nos permitirán analizar el comportamiento del modelo a través de gráficas como la matriz de confusión y las curvas de aprendizaje. Estas importaciones preparan el entorno completo para ejecutar todo el pipeline de principio a fin.




In [ ]:
!pip install -q scikit-learn seaborn matplotlib kaggle
try:
    import catboost
    from catboost import CatBoostClassifier, Pool
    HAS_CATBOOST = True
except Exception:
    HAS_CATBOOST = False
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# **3. Definición de la Clase del Modelo**

En esta sección definimos la clase encargada de todo el proceso de construcción del modelo de predicción, incluyendo el preprocesamiento, la ingeniería de características, el entrenamiento y la generación de predicciones finales. Esta clase agrupa de manera modular y organizada todas las funciones necesarias para limpiar los datos, transformar variables categóricas, realizar codificación basada en el rendimiento histórico, crear nuevas características socioeconómicas y educativas, entrenar el modelo CatBoost y producir los resultados en el formato requerido por Kaggle. Centralizar el pipeline dentro de una clase permite mantener el código más limpio, reutilizable y fácil de depurar, además de facilitar la ejecución tanto del modelo de validación como del modelo final entrenado con todos los datos disponibles.


In [ ]:
class StudentPerformanceClassifier:

    def __init__(self):
        self.model = None
        self.feature_importance = None
        self.target_encodings = {}
        self.training_columns = []
        self.categorical_features = []
        self.evals_result = None
        self.df_train = None
        self.class_weights = None

    def _encode_non_numeric(self, df):
        for col in df.columns:
            if not pd.api.types.is_numeric_dtype(df[col]):
                df[col] = pd.Categorical(df[col]).codes
        return df

    # ... other methods ...

    def _preprocess_common(self, df):
        cols_to_drop = [
            "ID",
            "INDICADOR_1",
            "INDICADOR_2",
            "INDICADOR_3",
            "INDICADOR_4",
            "F_TIENEINTERNET.1"
        ]

        X = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore").copy()

        y = None
        if "RENDIMIENTO_GLOBAL" in X.columns:
            y = X.pop("RENDIMIENTO_GLOBAL")

        for col in X.select_dtypes(include=["object"]).columns:
            X[col] = X[col].fillna("No reporta")

        if "PERIODO_ACADEMICO" in X.columns:
            X["PERIODO_ACADEMICO"] = X["PERIODO_ACADEMICO"].astype(str)

        X = self.create_education_features(X)
        X = self.create_economic_features(X)

        if not ('HAS_CATBOOST' in globals() and HAS_CATBOOST):
            X = self._encode_non_numeric(X)

        return X, y

    def preprocess_data(self, data, validation_mode=True):
        X, y = self._preprocess_common(data)

        class SklearnPool:
            def __init__(self, X, y, cat_features=None):
                self._X = X
                self._y = y
                self._cat = cat_features or []
            def get_features(self):
                return self._X
            def get_label(self):
                return np.array(self._y) if y is not None else None

        if validation_mode:
            print("Modo: Validación (dividiendo datos)")

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42, stratify=y
            )

            train_target_df = X_train.copy()
            train_target_df["RENDIMIENTO_GLOBAL"] = y_train

            X_train = self.create_location_program_features(X_train, train_target_df)
            X_test = self.create_location_program_features(X_test)

            if not ('HAS_CATBOOST' in globals() and HAS_CATBOOST):
                X_train = self._encode_non_numeric(X_train)
                X_test = self._encode_non_numeric(X_test)

            self.df_train = X_train.copy()
            self.training_columns = X_train.columns
            self.categorical_features = X_train.select_dtypes(include=["object"]).columns.tolist()

            class_counts = y_train.value_counts()
            total = class_counts.sum()
            weights = {cls: total / (len(class_counts) * cnt) for cls, cnt in class_counts.items()}
            self.class_weights = weights

            if 'HAS_CATBOOST' in globals() and HAS_CATBOOST:
                return (
                    Pool(X_train, y_train, cat_features=self.categorical_features)
,
                    Pool(X_test, y_test, cat_features=self.categorical_features)
                )
            else:
                return (
                    SklearnPool(X_train, y_train, self.categorical_features)
,
                    SklearnPool(X_test, y_test, self.categorical_features)
                )

        else:
            print("Modo: Entrenamiento final")

            df_target = pd.concat([X, y.rename("RENDIMIENTO_GLOBAL")], axis=1)
            X = self.create_location_program_features(X, df_target)

            if not ('HAS_CATBOOST' in globals() and HAS_CATBOOST):
                X = self._encode_non_numeric(X)

            self.df_train = X.copy()
            self.training_columns = X.columns
            self.categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

            class_counts = y.value_counts()
            total = class_counts.sum()
            self.class_weights = {cls: total / (len(class_counts) * cnt) for cls, cnt in class_counts.items()}

            if 'HAS_CATBOOST' in globals() and HAS_CATBOOST:
                return Pool(X, y, cat_features=self.categorical_features)
            else:
                return SklearnPool(X, y, self.categorical_features)

# **4. Uso de la Clase**

En esta sección utilizamos la clase previamente definida para ejecutar el flujo completo de validación del modelo, comenzando por cargar el dataset de entrenamiento, aplicar todo el proceso de preprocesamiento y generar los conjuntos de datos internos que CatBoost requiere (train_pool y test_pool). Luego se instancia el clasificador, se divide la información en datos de entrenamiento y validación y se entrena el modelo activando técnicas como early stopping gracias al uso del eval_pool. Este paso nos permite verificar el comportamiento real del modelo antes de entrenar la versión final, analizar su rendimiento, revisar las curvas de aprendizaje y validar que todos los componentes del pipeline —como la ingeniería de características, las codificaciones y los parámetros del modelo— están funcionando correctamente antes de continuar con el entrenamiento final y la creación del archivo de envío para Kaggle.


In [ ]:
# Cargar datos con búsqueda robusta de rutas
import os, glob, zipfile

def find_file(filename):
    candidates = [
        '.',
        os.getcwd(),
        os.path.expanduser('~'),
        os.path.join(os.path.expanduser('~'), 'Downloads'),
        os.path.dirname(os.path.abspath('.')),
    ]
    for d in candidates:
        p = os.path.join(d, filename)
        if os.path.exists(p):
            return p
    # Búsqueda recursiva en Descargas (acotada)
    dl = os.path.join(os.path.expanduser('~'), 'Downloads')
    try:
        matches = glob.glob(os.path.join(dl, '**', filename), recursive=True)
        if matches:
            return matches[0]
    except Exception:
        pass
    return None

# Intentar descomprimir si hay zip de Kaggle
def maybe_unzip_kaggle_zip():
    dl = os.path.join(os.path.expanduser('~'), 'Downloads')
    zips = glob.glob(os.path.join(dl, 'udea*pruebas**.zip')) + glob.glob('udea*pruebas**.zip')
    for z in zips:
        try:
            with zipfile.ZipFile(z, 'r') as zf:
                zf.extractall(os.path.dirname(z))
        except Exception:
            continue

TRAIN_PATH = os.environ.get('SP_TRAIN_PATH') or find_file('train.csv')
TEST_PATH = os.environ.get('SP_TEST_PATH') or find_file('test.csv')

if TRAIN_PATH is None or TEST_PATH is None:
    maybe_unzip_kaggle_zip()
    TRAIN_PATH = TRAIN_PATH or find_file('train.csv')
    TEST_PATH = TEST_PATH or find_file('test.csv')

if TRAIN_PATH is None:
    raise FileNotFoundError("No se encontró 'train.csv'. Coloca el archivo en la misma carpeta del notebook o en Descargas.")

print(f"Cargando train.csv desde: {TRAIN_PATH}")
data = pd.read_csv(TRAIN_PATH)

Cargando train.csv desde: .\train.csv


In [ ]:
# Instanciar el clasificador
print("=== Iniciando Fase de Validación ===")
classifier = StudentPerformanceClassifier()

=== Iniciando Fase de Validación ===


In [ ]:
# Preprocesamiento en modo validación
train_pool, test_pool = classifier.preprocess_data(
    data,
    validation_mode=True
)

Modo: Validación (dividiendo datos)


In [ ]:
# Entrenamiento del modelo con eval_pool para activar early stopping
print("\n=== Entrenando modelo de validación... ===")
classifier.train_model(train_pool, eval_pool=test_pool)
print("=== Entrenamiento completado ===")


=== Entrenando modelo de validación... ===


ValueError: could not convert string to float: 'CONTADURIA PUBLICA'

In [ ]:
# Graficar curvas de aprendizaje
print("\n=== Curvas de aprendizaje ===")
classifier.plot_learning_curve()

In [ ]:
# Ver la forma final de los datos preprocesados
classifier.df_train.head()


# **5. Evaluación del Modelo en Fase de Validación**

En esta sección evaluamos el rendimiento del modelo entrenado durante la fase de validación, utilizando exclusivamente el conjunto de datos reservado como test interno para medir la capacidad real de generalización. A partir de las predicciones generadas por el modelo, calculamos métricas clave como precisión, recall, f1-score y accuracy, lo que nos permite identificar el comportamiento del clasificador en cada una de las clases objetivo del rendimiento académico. Asimismo, construimos y visualizamos la matriz de confusión para detectar patrones de error, especialmente la forma en que el modelo distingue entre categorías como medio-alto y medio-bajo, que suelen presentar mayor confusión. Esta evaluación es fundamental para validar que la ingeniería de características, las codificaciones y los parámetros del modelo están funcionando correctamente antes de proceder al entrenamiento final con todos los datos disponibles.


In [ ]:
report, cm = classifier.evaluate_model(test_pool)

# Nombres reales de las clases
label_names = ['bajo', 'medio-bajo', 'medio-alto', 'alto']

print("\n=== Reporte de Clasificación ===")
print(report)

In [ ]:
print("\n=== Matriz de Confusión ===")
plt.figure(figsize=(8, 6))

sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=label_names,
    yticklabels=label_names
)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# **6. Entrenamiento Final y Generación de Predicciones para Kaggle**

En esta etapa realizamos el entrenamiento final del modelo utilizando el 100% de los datos disponibles en el archivo *train.csv*, sin dividir en validación, con el fin de maximizar la capacidad de aprendizaje del clasificador antes de generar las predicciones oficiales para el reto de Kaggle. Primero se preprocesa nuevamente todo el dataset completo, aplicando la misma ingeniería de características, codificaciones y transformaciones que se emplearon en la fase de validación, garantizando consistencia entre el modelo final y las pruebas anteriores. Una vez entrenado el modelo final, cargamos el archivo *test.csv*, aplicamos el preprocesamiento correspondiente y generamos las etiquetas de rendimiento para cada estudiante, produciendo finalmente el archivo *my_submission.csv* en el formato exacto exigido por la plataforma. Este archivo será el que posteriormente se enviará a Kaggle para obtener la puntuación del modelo en los datos ocultos del concurso.

In [ ]:
print("\n\n--- Iniciando Fase de Entrenamiento Final ---")

# Cargar test de Kaggle
X_test = pd.read_csv('test.csv')

In [ ]:
# Crear nuevo clasificador limpio
final_classifier = StudentPerformanceClassifier()

In [ ]:
# Preprocesar TODOS los datos de entrenamiento (sin validación)
final_train_pool = final_classifier.preprocess_data(
    data,
    validation_mode=False
)

In [ ]:
# Entrenar el modelo final
print("\n--- Entrenando modelo final... ---")
final_classifier.train_model(final_train_pool)
print("--- Entrenamiento final completado. ---")

In [ ]:
# ============================================================
# GENERAR Y GUARDAR PREDICCIONES PARA KAGGLE
# ============================================================

results = final_classifier.save_predictions(
    X_test,
    'my_submission.csv'
)

In [ ]:
submission = final_classifier.save_predictions(X_test, "my_submission.csv")
submission.head()


In [ ]:
results

In [ ]:
print("\n=== Conteo de clases reales (train) ===")
print(data['RENDIMIENTO_GLOBAL'].value_counts())

# **7. Envío del archivo a Kaggle**

En esta sección realizamos el envío oficial del archivo *my_submission.csv* a Kaggle. Una vez entrenado el modelo final y generadas las predicciones, usamos la interfaz de línea de comandos de Kaggle para cargar el archivo en la competencia y obtener la puntuación correspondiente en los datos ocultos del reto. Este paso completa el pipeline, permitiendo evaluar el desempeño real del modelo en el entorno competitivo.

In [ ]:
!head my_submission.csv
!kaggle competitions list

In [ ]:
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f my_submission.csv -m "versión final catboost"